<a href="https://colab.research.google.com/github/ayseakbaba/CS-Repos/blob/main/Recommender_TOP_N_Movie_Using_Surprise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 5.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357272 sha256=514852ff905ea89b0054b85e36ef463afc412363b2fc84ad9d02dd15af283807
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [7]:
data = Dataset.load_builtin('ml-100k')


In [9]:
data_folder = os.path.expanduser('~/.surprise_data/ml-100k')
print(f"Dataset ml-100k is stored in: {data_folder}")

Dataset ml-100k is stored in: /root/.surprise_data/ml-100k


In [10]:
%cd /root/.surprise_data/ml-100k

/root/.surprise_data/ml-100k


In [12]:
%cd ml-100k

/root/.surprise_data/ml-100k/ml-100k


In [13]:
%ls

allbut.pl  u1.base  u2.test  u4.base  u5.test  ub.base  u.genre  u.occupation
mku.sh     u1.test  u3.base  u4.test  ua.base  ub.test  u.info   u.user
README     u2.base  u3.test  u5.base  ua.test  u.data   u.item


In [14]:
import os
import pandas as pd
from surprise import Dataset, SVD, SVDpp, NMF, SlopeOne
from surprise import accuracy
from surprise.model_selection import train_test_split

# Veri kümesini yükleme
data = Dataset.load_builtin('ml-100k')

# MovieLens verilerinin bulunduğu klasörü bulma
data_folder = os.path.expanduser('/root/.surprise_data/ml-100k/ml-100k/')

# Film bilgilerini içeren dosyayı pandas ile yükleyin
movie_info_path = os.path.join(data_folder, 'u.item')
movie_info = pd.read_csv(movie_info_path, sep='|', encoding='latin-1', header=None)
movie_info.columns = ['movie_id', 'movie_title'] + [str(i) for i in range(22)]  # Sadece ilk iki sütunu kullanacağız
movie_dict = dict(zip(movie_info['movie_id'], movie_info['movie_title']))

# Eğitim ve test setlerini oluşturma
trainset, testset = train_test_split(data, test_size=0.2)

# Algoritmaları tanımlama
algo1 = SVD()
algo2 = SVDpp()
algo3 = NMF()
algo4 = SlopeOne()

# Algoritmaları eğitim seti üzerinde eğitme
algo1.fit(trainset)
algo2.fit(trainset)
algo3.fit(trainset)
algo4.fit(trainset)



In [16]:
# Kullanıcıya uygun önerilerde bulunma
user_id = str(196)  # Öneri yapılacak kullanıcı ID'si
item_ids = [iid for (uid, iid, _) in testset if uid == user_id]  # Kullanıcının izleyebileceği filmleri seç

def get_recommendations(algo, user_id, item_ids, movie_dict, top_n=5):
    predictions = [algo.predict(user_id, iid) for iid in item_ids]
    predictions.sort(key=lambda x: x.est, reverse=True)
    top_predictions = predictions[:top_n]
    # Film isimlerini kullanarak öneriler oluştur
    return [(movie_dict[int(pred.iid)], int(pred.est)) for pred in top_predictions]

# Her algoritma için film önerileri
print("SVD Algoritmasına Göre Öneriler:")
print(get_recommendations(algo1, user_id, item_ids, movie_dict))

print("\nSVD++ Algoritmasına Göre Öneriler:")
print(get_recommendations(algo2, user_id, item_ids, movie_dict))

print("\nNMF Algoritmasına Göre Öneriler:")
print(get_recommendations(algo3, user_id, item_ids, movie_dict))

print("\nSlopeOne Algoritmasına Göre Öneriler:")
print(get_recommendations(algo4, user_id, item_ids, movie_dict))


SVD Algoritmasına Göre Öneriler:
[('Kolya (1996)', 4), ('Boogie Nights (1997)', 4), ('Mrs. Brown (Her Majesty, Mrs. Brown) (1997)', 3), ('Raising Arizona (1987)', 3), ('Groundhog Day (1993)', 3)]

SVD++ Algoritmasına Göre Öneriler:
[('Mrs. Brown (Her Majesty, Mrs. Brown) (1997)', 4), ('Kolya (1996)', 4), ('Raising Arizona (1987)', 3), ('Mighty Aphrodite (1995)', 3), ('Boogie Nights (1997)', 3)]

NMF Algoritmasına Göre Öneriler:
[('Boogie Nights (1997)', 5), ('Mrs. Brown (Her Majesty, Mrs. Brown) (1997)', 4), ('Kolya (1996)', 4), ('Raising Arizona (1987)', 4), ('Van, The (1996)', 4)]

SlopeOne Algoritmasına Göre Öneriler:
[('Raising Arizona (1987)', 4), ('Mrs. Brown (Her Majesty, Mrs. Brown) (1997)', 4), ('Kolya (1996)', 4), ('Groundhog Day (1993)', 3), ('Boogie Nights (1997)', 3)]
